### Battle Of the Neighborhoods

In [1]:
# import all libraries required
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize 
import json

from geopy.geocoders import Nominatim 
import requests 

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!pip install folium
import folium
from bs4 import BeautifulSoup

print("Libraries  have been imported")

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly require

In [2]:
# The code was removed by DSX for sharing.

### Step 1 : Load the list of cities

In [3]:
default_list = ["Lucknow", "Varanasi", "Bareilly","Noida"]
input_argument = input("Enter your choice of cities from india (Seperated by a comma [,]) or press c and enter")
if input_argument == 'c':
    list_to_use = default_list
else:
    list_to_use = input_argument.split(',')

Enter your choice of cities from india (Seperated by a comma [,]) or press c and enterc


In [4]:
list_to_use

['Lucknow', 'Varanasi', 'Bareilly', 'Noida']

In [5]:
#define a function for extra
def get_postcode_coordinates(api_key, postcode, country='India', verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?address={},{}&key={}'.format(postcode, country,api_key)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return lat, lon
    except:
        return None, None

In [6]:
lat, lon = get_postcode_coordinates(API_key, 'Lucknow')
print('Lat:', lat, 'Lon:', lon)

Lat: 26.8466937 Lon: 80.94616599999999


In [7]:
location_dictionary ={}
for city in list_to_use:
    lat, lon = get_postcode_coordinates(API_key, city)
    #also store the data in a dictionary
    location_dictionary[city] = lat,lon
    print('Lat:', lat, 'Lon:', lon)

Lat: 26.8466937 Lon: 80.94616599999999
Lat: 25.3176452 Lon: 82.9739144
Lat: 28.3670355 Lon: 79.4304381
Lat: 28.5355161 Lon: 77.3910265


In [8]:
location_dictionary

{'Bareilly': (28.3670355, 79.4304381),
 'Lucknow': (26.8466937, 80.94616599999999),
 'Noida': (28.5355161, 77.3910265),
 'Varanasi': (25.3176452, 82.9739144)}

### Step 2 : Access the Foursquare API to locate the coffee shops in the region

In [9]:
# The code was removed by DSX for sharing.

In [10]:
#dataframe for storing he related data from cities
df = pd.DataFrame(columns =['city','shop_name','latitude','longitude'])
#df['city'] = location_dictionary.keys()
df

,city,shop_name,latitude,longitude


In [11]:
def get_top_venues(loc_name, lat, lon, client_id, client_secret,query="coffee",intent="browse", radius=5000, limit=100):
    version = '20180605'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&query={}&ll={},{}&intent={}&radius={}&limit={}'.format(
        client_id, client_secret, version,query,lat, lon,intent,radius, limit)
    
    venues = requests.get(url).json()['response']['venues']
    
    #temporary dataframe
    df_temp = pd.DataFrame(columns =['city','shop name','latitude','longitude'])
    
    #from all these venues extract the name and coords
    for i in range(len(venues)):
        df_temp.loc[i] = [loc_name,venues[i]["name"],venues[i]['location']['lat'],venues[i]['location']['lng']]
    return df_temp
        #shop_name.append(venues[i]["name"])
        #shop_address.append(venues[i]['location']["address"])
        #shop_latitude.append(venues[i]['location']['lat'])
        #shop_longitude.append(venues[i]['location']['lng'])
        
        
#    venues = [(loc_name,
#               lat,
#               lon,
#               row['venue']['name'],
#               row['venue']['location']['lat'],
#               row['venue']['location']['lng'],
#               get_category_type(row)) for row in results]

In [12]:
for loc_name, coords in location_dictionary.items():
    df = pd.concat([df,get_top_venues(loc_name, coords[0],coords[1], CLIENT_ID, CLIENT_SECRET)])

In [13]:
df

,city,latitude,longitude,shop name,shop_name
0,Noida,28.519199,77.380990,Cafe Coffee Day (ATS Village),NaN
1,Noida,28.516178,77.383467,cafe coffee day,NaN
2,Noida,28.515666,77.371611,Costa Coffee (Jaypee Hospital),NaN
3,Noida,28.501340,77.401323,Cafe Coffee Day,NaN
4,Noida,28.499731,77.404219,Cafe Coffee Day,NaN
5,Noida,28.534442,77.346292,Coffee World,NaN
6,Noida,28.567270,77.360541,coffee shop,NaN
7,Noida,28.537571,77.342350,Coffee Day xpress,NaN
8,Noida,28.570511,77.362161,Cafe Coffee Day Sector 50 Noida,NaN
0,Varanasi,25.323012,82.990287,cafe coffee day,NaN


### Figure out which city has the most consumption

In [14]:
#here we calculate the top 3 cities with max number of coffee shops
top3cities_df = df.city.value_counts()[:3]
top3cities = df.city.value_counts()[:3].index.tolist()
top3cities

['Lucknow', 'Noida', 'Varanasi']

In [15]:
df[df['city']=='Lucknow'].latitude

0    26.847243
1    26.855659
2    26.849581
3    26.881280
4    26.871536
5    26.873806
6    26.824853
7    26.857688
8    26.846255
Name: latitude, dtype: float64

### latitude and longitude from desired city

## predict the ideal coordinates in the city of your choice

#### ideal_location in each city is to be calculated as the mean location of the shops in the city

In [16]:
listing=[]
for cityname in top3cities:
    desired_city_coords = df[df['city']==cityname].latitude.values,df[df['city']==cityname].longitude.values
    listing.append((np.mean(desired_city_coords[0]),np.mean(desired_city_coords[1])))
    print("Optimum location in {} will be  :{}  ".format(cityname,(np.mean(desired_city_coords[0]),np.mean(desired_city_coords[1]))))

Optimum location in Lucknow will be  :(26.856433628903989, 80.958061639172641)  
Optimum location in Noida will be  :(28.529100839553582, 77.372550358852493)  
Optimum location in Varanasi will be  :(25.319060974773311, 82.994605457481683)  


### Now we need a search call for available properties in the region

#### we have used zillow api for retrieving property data

In [17]:
listing

[(26.856433628903989, 80.958061639172641),
 (28.529100839553582, 77.372550358852493),
 (25.319060974773311, 82.994605457481683)]

In [18]:
# The code was removed by DSX for sharing.

In [ ]:
locationforshop=[]

for _ in listing:
    xval = _[0]
    yval = _[1]
    url ="http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id={}&zpid={}&ll={},{}".format(zwsid,zpid,xval,yval)
    locationforshop[(requests.get(url).json()['simplex']["owner"])] = requests.get(url).json()['simplex']["info"],requests.get(url).json()['simplex']["coords"]
    #locationforshop is a dictionary with values as tuple of tuple (nested tuple)

In [ ]:
for i in range(len(listing)):
    map_cluster = folium.Map([listing[i][0],listing[i][1]], zoom_start=12)
    colors = ['yellow', 'green', 'blue', 'red', 'magenta']
    for i in range(len(locationforshop)):
        for lat, lon in locationforshop[1][i]:
            label = ' {} '.format(locationforshop["info"])
            folium.CircleMarker([lat, lon], radius=7, popup=folium.Popup(label), color=colors[cluster], fill=True, fill_color=colors[cluster], fill_opacity=0.7).add_to(map_clusters)
       
        map_cluster